<h1>Projeto SuperComputação 2024.2</h1>

**Análise de Redes Sociais: encontrando a clique máxima em um grafo.**

*Caio Ortega Boa*

![Representação gráfica de um clique](VR_complex.svg.png)

<h2>Introdução</h2>

O objetivo deste projeto é implementar e comparar diferentes abordagens para encontrar a clique máxima em um grafo, explorando técnicas de computação paralela para aumentar a eficiência e heurísticas para reduzir o tempo de execução.

Para tal foi fornecido um arquivo de geração de grafos, o qual foi utilizado com valores de vértice [100,150,160,170,180] e 0.7 de probabilidade de aresta.

Todas as execuções foram realizadas no cluster fornecido em sala considerando números de CPUs equivalentes para cada execução, possibilitando uma comparação justa entre as diferentes abordagens.

<h2>Soluções Propostas</h2>

- **Solução Exaustiva**

    Solução que testa todas as possibilidades de cliques em um grafo, retornando o clique de maior tamanho.


- **Solução Exaustiva com Poda**

    Solução que testa todas as possibilidades eliminando cliques que não podem ser maiores que o clique atual e ordenando os vértices de acordo com o grau.

- **Solução com OpenMP**

    Solução que utiliza a biblioteca OpenMP para paralelizar a busca de cliques, enquanto aproveita dos benefícios da heurística de poda e ordenamento.
    
- **Solução com OpenMPI**

    Solução que utiliza a biblioteca OpenMPI para paralelizar a busca de cliques, enquanto aproveita dos benefícios da heurística de poda e ordenamento.

<h3>Solução Exaustiva</h3>

In [ ]:
%%writefile Exaustiva.cpp

#include <iostream>
#include <vector>
#include <algorithm>
#include <chrono>
#include <string>
#include <fstream>
#include <iomanip>

// Função para carregar o grafo a partir de um arquivo
std::vector<std::vector<int>> CarregarGrafo(const std::string& caminhoArquivo, int& totalVertices) {
    std::ifstream entrada(caminhoArquivo);
    if (!entrada.is_open()) {
        std::cerr << "Falha ao abrir o arquivo " << caminhoArquivo << std::endl;
        return {};
    }

    int totalArestas;
    entrada >> totalVertices >> totalArestas;

    // Cria matriz de adjacência preenchida com zeros
    std::vector<std::vector<int>> matrizAdjacencia(totalVertices, std::vector<int>(totalVertices, 0));

    for (int i = 0; i < totalArestas; ++i) {
        int origem, destino;
        entrada >> origem >> destino;
        matrizAdjacencia[origem - 1][destino - 1] = 1;
        matrizAdjacencia[destino - 1][origem - 1] = 1;
    }

    entrada.close();
    return matrizAdjacencia;
}

// Função recursiva para explorar todas as possibilidades e identificar a maior clique
void ExplorarClique(const std::vector<std::vector<int>>& matrizAdjacencia, std::vector<int>& cliqueAtual, 
                    std::vector<int>& melhorClique, int totalVertices, int verticeInicial) {
    if (cliqueAtual.size() > melhorClique.size()) {
        melhorClique = cliqueAtual;
    }

    for (int vertice = verticeInicial; vertice < totalVertices; ++vertice) {
        bool todosConectados = true;

        for (int vizinho : cliqueAtual) {
            if (matrizAdjacencia[vizinho][vertice] == 0) {
                todosConectados = false;
                break;
            }
        }

        if (todosConectados) {
            cliqueAtual.push_back(vertice);
            ExplorarClique(matrizAdjacencia, cliqueAtual, melhorClique, totalVertices, vertice + 1);
            cliqueAtual.pop_back();
        }
    }
}

// Função principal para encontrar a maior clique no grafo
std::vector<int> ObterMaiorClique(const std::vector<std::vector<int>>& matrizAdjacencia, int totalVertices) {
    std::vector<int> melhorClique;
    std::vector<int> cliqueAtual;
    ExplorarClique(matrizAdjacencia, cliqueAtual, melhorClique, totalVertices, 0);
    return melhorClique;
}

int main() {
    int totalVertices;
    std::string caminhoArquivo = "grafo170.txt";

    // Carrega o grafo do arquivo especificado
    std::vector<std::vector<int>> matrizAdjacencia = CarregarGrafo(caminhoArquivo, totalVertices);

    auto inicioExecucao = std::chrono::high_resolution_clock::now();

    // Obtém a maior clique utilizando busca exaustiva
    std::vector<int> melhorClique = ObterMaiorClique(matrizAdjacencia, totalVertices);

    auto tempoExecucao = std::chrono::duration_cast<std::chrono::duration<double>>(
                            std::chrono::high_resolution_clock::now() - inicioExecucao).count();

    // Exibe os resultados
    std::cout << "Maior clique -> ";
    for (int vertice : melhorClique) {
        std::cout << vertice + 1 << " ";
    }
    std::cout << "\n";

    std::cout << "Tamanho da maior clique -> " << melhorClique.size() << "\n";

    std::cout << std::fixed << std::setprecision(6);
    std::cout << "Tempo de execução -> " << tempoExecucao << " segundos\n";

    return 0;
}

In [ ]:
%%writefile submit_exaust.sh

#!/bin/bash
#SBATCH --job-name=exaust
#SBATCH --output=output_exaust100.txt
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=4
#SBATCH --mem-per-cpu=1Gb
#SBATCH --time=00:15:00
#SBATCH --partition=espec

./Exaustiva

In [ ]:
!g++ Exaustiva.cpp -o Exaustiva
!sbatch submit_exaust.sh

<h3>Solução Exaustiva com Poda</h3>

In [ ]:
%%writefile ExaustivaPoda.cpp

#include <iostream>
#include <vector>
#include <algorithm>
#include <chrono>
#include <fstream>
#include <iomanip>
#include <numeric> // Para std::iota

// Função para ler o grafo a partir de um arquivo
std::vector<std::vector<int>> LerGrafo(const std::string& nomeArquivo, int& numVertices) {
    std::ifstream arquivo(nomeArquivo);
    if (!arquivo.is_open()) {
        std::cerr << "Erro ao abrir o arquivo " << nomeArquivo << std::endl;
        return {};
    }

    int numArestas;
    arquivo >> numVertices >> numArestas;

    std::vector<std::vector<int>> grafo(numVertices, std::vector<int>(numVertices, 0));

    for (int i = 0; i < numArestas; ++i) {
        int u, v;
        arquivo >> u >> v;
        grafo[u - 1][v - 1] = 1;
        grafo[v - 1][u - 1] = 1;
    }

    return grafo;
}

// Função recursiva com poda para encontrar a clique máxima
void BuscarCliqueComPoda(const std::vector<std::vector<int>>& grafo, std::vector<int>& atualClique, 
                         std::vector<int>& cliqueMaxima, std::vector<int>& candidatos, int& maxTamanho) {
    if (atualClique.size() + candidatos.size() <= maxTamanho) return;

    while (!candidatos.empty()) {
        int v = candidatos.back();
        candidatos.pop_back();

        std::vector<int> novosCandidatos;
        for (int u : candidatos) {
            if (grafo[v][u] == 1) {
                novosCandidatos.push_back(u);
            }
        }

        atualClique.push_back(v);
        if (atualClique.size() > cliqueMaxima.size()) {
            cliqueMaxima = atualClique;
            maxTamanho = cliqueMaxima.size();
        }

        BuscarCliqueComPoda(grafo, atualClique, cliqueMaxima, novosCandidatos, maxTamanho);
        atualClique.pop_back();
    }
}

// Função principal para encontrar clique máxima com poda
std::vector<int> EncontrarCliqueMaximaHeuristica(const std::vector<std::vector<int>>& grafo) {
    int numVertices = grafo.size();
    std::vector<int> candidatos(numVertices);
    std::iota(candidatos.begin(), candidatos.end(), 0); // Inicializa com 0, 1, ..., numVertices-1

    std::vector<int> cliqueMaxima;
    std::vector<int> atualClique;
    int maxTamanho = 0;

    BuscarCliqueComPoda(grafo, atualClique, cliqueMaxima, candidatos, maxTamanho);
    return cliqueMaxima;
}

int main() {
    int numVertices;
    std::string nomeArquivo = "grafo180.txt";

    std::vector<std::vector<int>> grafo = LerGrafo(nomeArquivo, numVertices);
    if (grafo.empty()) return 1;

    auto inicio = std::chrono::high_resolution_clock::now();
    std::vector<int> cliqueMaxima = EncontrarCliqueMaximaHeuristica(grafo);
    auto fim = std::chrono::high_resolution_clock::now();

    double duracao = std::chrono::duration_cast<std::chrono::duration<double>>(fim - inicio).count();

    std::cout << "Clique máxima encontrada: ";
    for (int v : cliqueMaxima) {
        std::cout << v + 1 << " ";
    }
    std::cout << "\nTamanho da clique máxima: " << cliqueMaxima.size() << std::endl;
    std::cout << std::fixed << std::setprecision(6) << "Tempo total de execução: " << duracao << " segundos\n";

    return 0;
}

In [ ]:
%%writefile submit_exaustpoda.sh

#!/bin/bash
#SBATCH --job-name=exaustpoda
#SBATCH --output=output_exaustpoda180.txt
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=4
#SBATCH --mem-per-cpu=1Gb
#SBATCH --time=00:15:00
#SBATCH --partition=espec

./ExaustivaPoda

In [ ]:
!g++ ExaustivaPoda.cpp -o ExaustivaPoda
!sbatch submit_exaustpoda.sh

<h3>Solução com OpenMP</h3>

In [ ]:
%%writefile OpenMP.cpp

#include <iostream>
#include <vector>
#include <algorithm>
#include <chrono>
#include <string>
#include <fstream>
#include <iomanip>
#include <omp.h>
#include <numeric> // Para std::iota e std::accumulate

// Restante do código permanece o mesmo...


// Função para carregar o grafo a partir de um arquivo
std::vector<std::vector<int>> CarregarGrafo(const std::string& caminhoArquivo, int& totalVertices) {
    std::ifstream entrada(caminhoArquivo);
    if (!entrada.is_open()) {
        std::cerr << "Falha ao abrir o arquivo " << caminhoArquivo << std::endl;
        return {};
    }

    int totalArestas;
    entrada >> totalVertices >> totalArestas;

    std::vector<std::vector<int>> matrizAdjacencia(totalVertices, std::vector<int>(totalVertices, 0));

    for (int i = 0; i < totalArestas; ++i) {
        int origem, destino;
        entrada >> origem >> destino;
        matrizAdjacencia[origem - 1][destino - 1] = 1;
        matrizAdjacencia[destino - 1][origem - 1] = 1;
    }

    entrada.close();
    return matrizAdjacencia;
}

// Função recursiva com melhorias e poda
void ExplorarCliqueComPoda(const std::vector<std::vector<int>>& matrizAdjacencia, 
                           std::vector<int>& cliqueAtual, std::vector<int>& melhorCliqueLocal, 
                           std::vector<int>& candidatos, int& maxCliqueSize) {
    if (cliqueAtual.size() + candidatos.size() <= maxCliqueSize) return;

    for (int i = 0; i < candidatos.size(); ++i) {
        int vertice = candidatos[i];

        std::vector<int> novosCandidatos;
        for (int j = i + 1; j < candidatos.size(); ++j) {
            if (matrizAdjacencia[vertice][candidatos[j]] == 1) {
                novosCandidatos.push_back(candidatos[j]);
            }
        }

        cliqueAtual.push_back(vertice);
        if (cliqueAtual.size() > melhorCliqueLocal.size()) {
            melhorCliqueLocal = cliqueAtual;
            #pragma omp critical
            maxCliqueSize = std::max(maxCliqueSize, (int)melhorCliqueLocal.size());
        }

        ExplorarCliqueComPoda(matrizAdjacencia, cliqueAtual, melhorCliqueLocal, novosCandidatos, maxCliqueSize);
        cliqueAtual.pop_back();
    }
}

// Função principal para encontrar a maior clique com heurísticas e OpenMP
std::vector<int> ObterMaiorClique(const std::vector<std::vector<int>>& matrizAdjacencia, int totalVertices) {
    std::vector<int> melhorClique;
    int maxCliqueSize = 0;

    // Pré-processamento: Ordenar vértices pelo grau (maior primeiro)
    std::vector<int> ordemVertices(totalVertices);
    std::iota(ordemVertices.begin(), ordemVertices.end(), 0);
    std::sort(ordemVertices.begin(), ordemVertices.end(), [&](int u, int v) {
        return std::accumulate(matrizAdjacencia[u].begin(), matrizAdjacencia[u].end(), 0) >
               std::accumulate(matrizAdjacencia[v].begin(), matrizAdjacencia[v].end(), 0);
    });

    #pragma omp parallel
    {
        std::vector<int> melhorCliqueLocal;
        std::vector<int> cliqueAtual;

        #pragma omp for schedule(dynamic) nowait
        for (int i = 0; i < totalVertices; ++i) {
            std::vector<int> candidatos;
            for (int j = i + 1; j < totalVertices; ++j) {
                if (matrizAdjacencia[ordemVertices[i]][ordemVertices[j]] == 1) {
                    candidatos.push_back(ordemVertices[j]);
                }
            }

            cliqueAtual.push_back(ordemVertices[i]);
            ExplorarCliqueComPoda(matrizAdjacencia, cliqueAtual, melhorCliqueLocal, candidatos, maxCliqueSize);
            cliqueAtual.pop_back();
        }

        #pragma omp critical
        {
            if (melhorCliqueLocal.size() > melhorClique.size()) {
                melhorClique = melhorCliqueLocal;
            }
        }
    }

    return melhorClique;
}

int main() {
    int totalVertices;
    std::string caminhoArquivo = "grafo180.txt";

    auto matrizAdjacencia = CarregarGrafo(caminhoArquivo, totalVertices);

    auto inicioExecucao = std::chrono::high_resolution_clock::now();

    std::vector<int> melhorClique = ObterMaiorClique(matrizAdjacencia, totalVertices);

    auto tempoExecucao = std::chrono::duration_cast<std::chrono::duration<double>>(
                            std::chrono::high_resolution_clock::now() - inicioExecucao).count();

    std::cout << "Maior clique -> ";
    for (int vertice : melhorClique) {
        std::cout << vertice + 1 << " ";
    }
    std::cout << "\n";

    std::cout << "Tamanho da maior clique -> " << melhorClique.size() << "\n";
    std::cout << std::fixed << std::setprecision(6);
    std::cout << "Tempo de execução -> " << tempoExecucao << " segundos\n";

    return 0;
}

In [ ]:
%%writefile submit_openmp.sh

#!/bin/bash
#SBATCH --job-name=openmp
#SBATCH --output=output_openmp180.txt
#SBATCH --ntasks=2
#SBATCH --cpus-per-task=2
#SBATCH --mem-per-cpu=1Gb
#SBATCH --time=00:15:00
#SBATCH --partition=espec

./OpenMP

In [ ]:
!g++ OpenMP.cpp -o OpenMP
!sbatch submit_openmp.sh

<h3>Solução com MPI</h3>

In [ ]:
%%writefile MPI.cpp

#include <mpi.h>
#include <iostream>
#include <vector>
#include <algorithm>
#include <fstream>
#include <iomanip>
#include <numeric>

std::vector<std::vector<int>> CarregarGrafo(const std::string& caminhoArquivo, int& totalVertices) {
    std::ifstream entrada(caminhoArquivo);
    if (!entrada.is_open()) {
        std::cerr << "Falha ao abrir o arquivo " << caminhoArquivo << std::endl;
        return {};
    }

    int totalArestas;
    entrada >> totalVertices >> totalArestas;

    std::vector<std::vector<int>> matrizAdjacencia(totalVertices, std::vector<int>(totalVertices, 0));

    for (int i = 0; i < totalArestas; ++i) {
        int origem, destino;
        entrada >> origem >> destino;
        matrizAdjacencia[origem - 1][destino - 1] = 1;
        matrizAdjacencia[destino - 1][origem - 1] = 1;
    }

    entrada.close();
    return matrizAdjacencia;
}

void ExplorarCliqueComPoda(const std::vector<std::vector<int>>& matrizAdjacencia, 
                           std::vector<int>& cliqueAtual, std::vector<int>& melhorCliqueLocal, 
                           std::vector<int>& candidatos, int& maxCliqueSize) {
    if (cliqueAtual.size() + candidatos.size() <= maxCliqueSize) return;

    for (int i = 0; i < candidatos.size(); ++i) {
        int vertice = candidatos[i];

        std::vector<int> novosCandidatos;
        for (int j = i + 1; j < candidatos.size(); ++j) {
            if (matrizAdjacencia[vertice][candidatos[j]] == 1) {
                novosCandidatos.push_back(candidatos[j]);
            }
        }

        cliqueAtual.push_back(vertice);
        if (cliqueAtual.size() > melhorCliqueLocal.size()) {
            melhorCliqueLocal = cliqueAtual;
            maxCliqueSize = std::max(maxCliqueSize, (int)melhorCliqueLocal.size());
        }

        ExplorarCliqueComPoda(matrizAdjacencia, cliqueAtual, melhorCliqueLocal, novosCandidatos, maxCliqueSize);
        cliqueAtual.pop_back();
    }
}

int main(int argc, char** argv) {
    MPI_Init(&argc, &argv);

    int totalVertices;
    std::string caminhoArquivo = "grafo180.txt";

    int rank, size;
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    MPI_Comm_size(MPI_COMM_WORLD, &size);

    std::vector<std::vector<int>> matrizAdjacencia;
    if (rank == 0) {
        matrizAdjacencia = CarregarGrafo(caminhoArquivo, totalVertices);
    }

    MPI_Bcast(&totalVertices, 1, MPI_INT, 0, MPI_COMM_WORLD);

    if (rank != 0) {
        matrizAdjacencia.resize(totalVertices, std::vector<int>(totalVertices, 0));
    }

    for (int i = 0; i < totalVertices; ++i) {
        MPI_Bcast(matrizAdjacencia[i].data(), totalVertices, MPI_INT, 0, MPI_COMM_WORLD);
    }

    std::vector<int> ordemVertices(totalVertices);
    std::iota(ordemVertices.begin(), ordemVertices.end(), 0);
    std::sort(ordemVertices.begin(), ordemVertices.end(), [&](int u, int v) {
        return std::accumulate(matrizAdjacencia[u].begin(), matrizAdjacencia[u].end(), 0) >
               std::accumulate(matrizAdjacencia[v].begin(), matrizAdjacencia[v].end(), 0);
    });

    int verticesPorProcessador = totalVertices / size;
    int inicio = rank * verticesPorProcessador;
    int fim = (rank == size - 1) ? totalVertices : inicio + verticesPorProcessador;

    std::vector<int> melhorCliqueLocal;
    double tempoInicio = MPI_Wtime();

    for (int i = inicio; i < fim; ++i) {
        std::vector<int> cliqueAtual = {ordemVertices[i]};
        std::vector<int> candidatos;

        for (int j = i + 1; j < totalVertices; ++j) {
            if (matrizAdjacencia[ordemVertices[i]][ordemVertices[j]] == 1) {
                candidatos.push_back(ordemVertices[j]);
            }
        }

        int maxCliqueSize = melhorCliqueLocal.size();
        ExplorarCliqueComPoda(matrizAdjacencia, cliqueAtual, melhorCliqueLocal, candidatos, maxCliqueSize);
    }

    double tempoExecucaoLocal = MPI_Wtime() - tempoInicio;

    if (rank != 0) {
        int tamanhoCliqueLocal = melhorCliqueLocal.size();
        MPI_Send(&tamanhoCliqueLocal, 1, MPI_INT, 0, 0, MPI_COMM_WORLD);
        MPI_Send(melhorCliqueLocal.data(), tamanhoCliqueLocal, MPI_INT, 0, 1, MPI_COMM_WORLD);
    } else {
        std::vector<int> melhorCliqueGlobal = melhorCliqueLocal;
        for (int i = 1; i < size; ++i) {
            int tamanhoCliqueRecebida;
            MPI_Recv(&tamanhoCliqueRecebida, 1, MPI_INT, i, 0, MPI_COMM_WORLD, MPI_STATUS_IGNORE);

            std::vector<int> cliqueRecebida(tamanhoCliqueRecebida);
            MPI_Recv(cliqueRecebida.data(), tamanhoCliqueRecebida, MPI_INT, i, 1, MPI_COMM_WORLD, MPI_STATUS_IGNORE);

            if (cliqueRecebida.size() > melhorCliqueGlobal.size()) {
                melhorCliqueGlobal = cliqueRecebida;
            }
        }

        double tempoExecucaoGlobal;
        MPI_Reduce(&tempoExecucaoLocal, &tempoExecucaoGlobal, 1, MPI_DOUBLE, MPI_MAX, 0, MPI_COMM_WORLD);

        std::cout << "Maior clique -> ";
        for (int vertice : melhorCliqueGlobal) {
            std::cout << vertice + 1 << " "; // Para ajustar caso a indexação comece de 1
        }
        std::cout << "\nTamanho da maior clique -> " << melhorCliqueGlobal.size() << "\n";
        std::cout << std::fixed << std::setprecision(6);
        std::cout << "Tempo total de execução -> " << tempoExecucaoGlobal << " segundos\n";
    }

    MPI_Finalize();
    return 0;
}

In [ ]:
%%writefile submit_mpi.sh

#!/bin/bash
#SBATCH --job-name=mpi
#SBATCH --output=output_mpi180.txt
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=4
#SBATCH --mem-per-cpu=1Gb
#SBATCH --time=00:15:00
#SBATCH --partition=espec

./MPI

In [ ]:
!mpic++ -o MPI MPI.cpp
!sbatch submit_mpi.sh

<h2> Análise de Resultados </h2>

![Gráfico de desempenho](GraficoTempos.png)

Como esperado a abordagem exaustiva foi a abordagem que apresentou pior desempenho, uma vez que testa todas as possibilidades de cliques, o que para grafos grandes se torna inviável, e como os testes foram todos realizados com tempo máximo de 15 minutos a abordagem exaustiva não conseguiu completar a execução para os grafos de 170 e 180 vértices.

A abordagem com OpenMP apresentou um desempenho superior ao da abordagem com MPI, o que possívelmente se deu pelo excesso de Overhead gerado pela comunicação realizada entre as máquinas do cluster.

O resultado inesperado obtido foi o da heurística de poda, que apresentou resultados superiores aos das abordagens paralelas, e isso pode ser explicado pelos seguintes fatores:

- *Overhead das abordagens paralelas*

    Todas as abordagens paralelas sofrem com certo overhead em sua execução, seja pela comunicação entre as máquinas, seja pela divisão de tarefas entre os processos. A heurística de poda, por ser uma abordagem sequencial, não sofre com esse overhead, porém isso não é o suficiente para explicar resultados superiores, uma vez que para grandes dados de entrada abordagens paralelas tendem a superar abordagens sequenciais mesmo considerando seus overheads.

- *Ordenação*

    A abordagem com poda é extremamente compátivel com dados ordenados, beneficiando principalmente a abordagem sequencial, que possuí fluxo contínuo e consome o grafo na ordem estipulada. Já as abordagens paralelas, por dividirem o grafo entre os processos, possuem certos problemas com os benefícios da ordenação, uma vez que processos separados podem consumir "galhos menos promissores" e a ordenação pode se perder em diferentes processos.

- *Poda*

    Na abordagem sequencial o valor da clique máxima é atualizado automaticamente, enquanto na abordagem com OpenMP esse depende de zonas críticas e em MPI depende da chegada dos valores no processo mestre.

- *Balanceamento de Carga*

    As divisões de carga em processos paralelos podem acarretar em atribuições de processos mais complexos para determinadas tarefas, reduzindo os benefícios da paralelização no problema.

Em suma pode-se dizer que o problema em questão se aproveita muito de estratégias que possuem melhor resposta para abordagens sequenciais, portanto não apresentando um ganho significativo com a paralelização.
Possívelmente com melhores implementações da paralelização, como uma divisão respeitando a ordenação entre as threads e uso dinâmico da heurísitica de poda dentro dos processos, os resultados das estratégias paralelas deveriam superar a estratégia sequencial.